In [ ]:
from qat.lang.AQASM import Program, QRoutine, RZ, RX, CNOT, H, X, Z
from qat.core import Observable, Term
from qat.qpus import get_default_qpu, PyLinalg
from qat.plugins import ScipyMinimizePlugin
from copy import deepcopy
import matplotlib.pyplot as plt
import math as m

# Hands-on: Algoritmo de Grover

## Sumário do Algoritmo de Grover

Seja $N=2^n$. Consideramos um registrador de $n$ qubits de modo que um dos elementos da base computacional é dado por $|x_0\rangle$. Definimos inicialmente uma condição inicial uniforme

$$|\psi_0\rangle =  \frac{1}{\sqrt{2^n}}\sum_{i=0}^{2^n-1} |x\rangle$$

Daí, definimos os operadores:

$$U_{f} |x\rangle|i\rangle=|x\rangle|i\oplus f(x)\rangle,$$
chamado de oráculo (codifica a função $f$ onde $f(x_0)=-1$ e $f(x)=0$ se $x\neq x_0$, onde $x_0$ é chamado elemento marcado).
O operador $G$ é
$$G = (2|\psi_0\rangle\langle \psi_0| - \mathrm{I}_N)\otimes \mathrm{I}_2, $$
chamado de operador de inversão em torno da média.

## Os passos do algoritmo

O algoritmo tem os seguintes passos (usando $n+1$ qubits)

1. Primeiro, preparar o estado inicial $|\psi_0\rangle|-\rangle$;

2. Aplicar o operador $U_f$;

3. Aplicar o operador $G$;

4. Repetir $t$ vezes os passos 2 e 3 onde

$$
t=\left\lfloor\frac{\pi}{4}\sqrt N\right\rfloor;
$$

5. Medir o primeiro registrador na base computacional.

A medição retorna o estado $x_0$ com probabilidade de aproximadamente $1 - \frac{1}{\sqrt{N}}$.

### Execício 1

(a) Implemente no $\verb|QLM|$ o circuito do operador $U_{f}$ no caso $N=4$ e $x_0=10$, ou seja, $f(10)=1$ e $f(j)=0$ se $ j\neq 10$. Mostre o circuito.

(b) Rode o circuito quando a entrada é $|10\rangle|0\rangle$ e verifique que a saída é $\ket{10}\ket{1}$, como previsto.

(c) Rode o circuito quando a entrada é $|10\rangle|-\rangle$ e verifique que a saída é $\big(-\ket{10}\ket{-}\big)$, como previsto.

(d) Rode o circuito quando a entrada é $|\psi_0\rangle|0\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{2}\sum_{i=0}^{3} |x\rangle$$
e verifique que a saída é uma superposição que tem $|1\rangle$ no segundo registrador apenas para o elemento marcado, como previsto.

(e) Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ e verifique que a saída é uma superposição com sinal invertido apenas para o elemento marcado, como previsto.

In [ ]:
num_qubits = 2
N = 2**num_qubits
targets = [(1,0)]

In [ ]:
# Exercício 1a
def oracle(num_qubits, targets):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)

    for e in targets:
        for i,val in enumerate(e):
            if val == 0:
                X(wires[i])
        X.ctrl(num_qubits)(wires, ancilla)
        for i,val in enumerate(e):
            if val == 0:
                X(wires[i])
    return routine

my_prog = Program()
Uf = oracle(num_qubits, targets)
my_qbits = my_prog.qalloc(num_qubits)
Uf(my_qbits)
circuit = my_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Exercício 1b
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
X(wires[0])

initial(Qr)
Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 1c
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
X(wires[0])
X(ancilla)
H(ancilla)

initial(Qr)
Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 1d
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)

initial(Qr)
Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 1e
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

### Exercício 2

O objetivo deste exercício é aprender implementar o operador
$$ A=( \mathrm{I}_N - 2|0\rangle\langle 0| ) {\otimes }\mathrm{I}_2$$
quando a entrada do segundo registrador é $|-\rangle$ (a sua implementação só funciona neste caso).

(a) Verifique através de uma análise algébrica que $A=U_f$, onde $f(0)=1$ e $f(j)=0$ caso contrário (sempre tome a entrada do segundo registrado como $|-\rangle$).

(b) Implemente no $\verb|QLM|$ o circuito do operador $A$ para $N=4$.  Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{2}\sum_{i=0}^{3} |x\rangle$$
e verifique que a saída é uma superposição com sinal invertido apenas quando o primeiro registrador é $|0\rangle$, como previsto.

In [ ]:
# Exercício 2b
def diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    for qubit in wires:
        X(qubit)
    X.ctrl(len(wires))(wires, ancilla)
    for qubit in wires:
        X(qubit)

    return routine

my_prog = Program()
A = diffusion(num_qubits)
my_qbits = my_prog.qalloc(num_qubits)
A(my_qbits)
circuit = my_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Exercício 2b
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
A(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

### Exercício 3

O objetivo deste exercício é aprender implementar o operador
$$ G=( \mathrm{I}_N - 2|\psi_0\rangle\langle\psi_0| ) {\otimes }\mathrm{I}_2$$
quando a entrada do segundo registrador é $|-\rangle$ (a sua implementação só funciona neste caso). Vimos na aula que $G=H^{\otimes n} A H^{\otimes n}$. Use este fato e a implementação de $A$ vista acima para fazer este exercício. [*NOTA: O operador $G$ neste exercício tem uma inversão de sinal. Isto não afeta o resultado do algoritmo, como visto em aula.*]

(a) Implemente no $\verb|QLM|$ o circuito do operador $G$ para $N=4$.  Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{2}\sum_{i=0}^{3} |x\rangle$$
e verifique que a saída é igual a $(-|\psi_0\rangle|-\rangle)$, como previsto.

(b) Escolha um vetor ortogonal a $|\psi_0\rangle$. Vamos denotar por $|\psi_0^\perp\rangle$. Rode o circuito quando a entrada é $|\psi_0^\perp\rangle|-\rangle$
e verifique que a saída é igual a entrada, como previsto.


In [ ]:
# Exercício 3a
def full_diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)
    for qubit in wires:
        H(qubit)
    for qubit in wires:
        X(qubit)
    # Do multi-controlled-Z gate
    X.ctrl(len(wires))(wires, ancilla)
    for qubit in wires:
        X(qubit)
    for qubit in wires:
        H(qubit)

    return routine

my_prog = Program()
G = full_diffusion(num_qubits)
my_qbits = my_prog.qalloc(num_qubits)
G(my_qbits)
circuit = my_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Exercício 3a
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 3b - Incompleto
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

### Exercício 4

Determine o número de repetições do algoritmo de Grover quando $N=4$.

In [ ]:
rep = int((m.pi/4)*m.sqrt(N))
rep

### Exercício 5

Implemente o algoritmo de Grover completo para $N=4$ e rode o algoritmo quando a entrada é $|0\rangle|-\rangle$ e o elemento marcado é $x_0=2$. Verifique se a saída da medição do primeiro registrador tem como saída a cadeia de bits correta.

In [ ]:
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial(Qr)
for i in range(rep):
    Uf(Qr)
    G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()
%qatdisplay circuit --svg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))


## Algoritmo de Grover com mais de 1 elemento marcado

### Execício 6

(a) Implemente no $\verb|QLM|$ o circuito do operador $U_{f}$ no caso $N=8$ tal que $f(x)=1$ se $x=110$ ou $x=011$ e $f(x)=0$ se caso contrário. Mostre o circuito.

(b) Rode o circuito quando a entrada é $|\psi_0\rangle|0\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{\sqrt{8}}\sum_{i=0}^{7} |x\rangle$$
e verifique que a saída é uma superposição que tem $|1\rangle$ no segundo registrador apenas para os elementos marcados, como previsto.

(c) Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ e verifique que a saída é uma superposição com sinal invertido apenas para os elementos marcados, como previsto.

In [ ]:
num_qubits = 3
N = 2**num_qubits
targets = [(1,1,0), (0,1,1)]

In [ ]:
# Ecercício 6a
def oracle_mult(num_qubits, targets):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    ancilla = routine.new_wires(1)
    routine.set_ancillae(ancilla)

    for e in targets:
        for i,val in enumerate(e):
            if val == 0:
                X(wires[i])
        X.ctrl(num_qubits)(wires, ancilla)
        for i,val in enumerate(e):
            if val == 0:
                X(wires[i])
    return routine

my_prog = Program()
Uf = oracle_mult(num_qubits, targets)
my_qbits = my_prog.qalloc(num_qubits)
Uf(my_qbits)
circuit = my_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Exercício 6b
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)

initial(Qr)
Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 6c
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

### Exercício 7

Seja
$$ G=( \mathrm{I}_N - 2\ket{\psi_0}\bra{\psi_0} ) {\otimes }\mathrm{I}_2$$
Tome $N=8$ e seja
$$|\psi_0\rangle =  \frac{1}{\sqrt{8}}\sum_{i=0}^{7} |x\rangle$$

(a) Implemente no $\verb|QLM|$ o circuito do operador $G$ para $N=8$.  Rode o circuito quando a entrada é $|\psi_0\rangle|-\rangle$ e verifique que a saída é igual a $(-|\psi_0\rangle|-\rangle)$, como previsto.

(b) Escolha um vetor ortogonal a $|\psi_0\rangle$. Vamos denotar por $|\psi_0^\perp\rangle$. Rode o circuito quando a entrada é $|\psi_0^\perp\rangle|-\rangle$
e verifique que a saída é igual a entrada, como previsto.

In [ ]:
# Exercício 7a
qprog = Program()
Qr = qprog.qalloc(num_qubits)
G = full_diffusion(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 7b - Incompleto
qprog = Program()
Qr = qprog.qalloc(num_qubits)
G = full_diffusion(num_qubits)

initial = QRoutine()
wires = initial.new_wires(num_qubits)
ancilla = initial.new_wires(1)
initial.set_ancillae(ancilla)
for q in wires:
    H(q)
X(ancilla)
H(ancilla)

initial(Qr)
G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

### Exercício 8

Determine o número de repetições do algoritmo de Grover quando $N=8$ com $m=2$ elementos marcados. Use a fórmula $\left\lfloor\frac{\pi}{4}\sqrt{\frac{N}{m}}\right\rfloor$.

In [ ]:
rep = int((m.pi/4)*m.sqrt(N/len(targets)))
rep

### Exercício 9

Implemente o algoritmo de Grover completo para $N=8$ com os elementos marcados $110$ e $011$. Rode o algoritmo quando a entrada é $|0\rangle|-\rangle$ e o elemento marcado é $x_0=2$. Verifique se a saída da medição do primeiro registrador é a cadeia de bits correta.

In [ ]:
qprog = Program()
Qr = qprog.qalloc(num_qubits)

initial(Qr)
for i in range(rep):
    Uf(Qr)
    G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()
%qatdisplay circuit --svg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

## Implementação econômica do algoritmo de Grover

### Execício 10

O objetivo do próximo exercício é implementar o algoritmo de Grover sem usar o segundo registrador. Vamos re-definir o operador $U_f$ usando apenas o primeiro registrador ($n$ qubits) como
$$U_{f} |x\rangle=(-1)^{f(x)}|x\rangle,$$
onde $f(x_0)=1$ e $f(x)=0$ se $x\neq x_0$ e $x_0=(i_1,...,i_n)_2$. Use o seguinte circuito para implementar $U_f$:

<img src="./Grover-econ.png" width="450px">

Note que $X^{i_1}=\text{I}$ se $i_1=0$ e $X^{i_1}=X$ se $i_1=1$.

(a) Implemente no $\verb|QLM|$ o circuito do operador $U_{f}$ no caso $N=8$ tal que $f(x)=1$ se $x=110$ e $f(x)=0$ caso contrário. Mostre o circuito.

(b) Rode o circuito quando a entrada é $|\psi_0\rangle$ onde
$$|\psi_0\rangle =  \frac{1}{\sqrt{8}}\sum_{i=0}^{7} |x\rangle$$
e verifique que a saída é uma superposição com sinal invertido apenas para o elemento marcado, como previsto.

(c) Implemente no $\verb|QLM|$ o circuito do operador $G$ no caso $N=8$ usando apenas 3 qubits. Use o fato de que $G=H^{\otimes n}U_{f_0}H^{\otimes n}$ quando $f_0(x)=1$ se $x=000$ e $f_0(x)=0$ se caso contrário. Mostre o circuito e cheque que o operador $G$ está implementado corretamente.

(d) Implemente no $\verb|QLM|$ o circuito do algoritmo de Grover no caso $N=8$ com $x_0=110$ usando apenas 3 qubits. Faça um histograma que mostre a probabilidade de sucesso com pelo menos 3 casas decimais. Compare com o valor exato de $121/128$.

In [ ]:
num_qubits = 3
N = 2**num_qubits
targets = [(1,1,0)]
rep = int((m.pi/4)*m.sqrt(N/len(targets)))

In [ ]:
# Exercício 10a
def economic_oracle(num_qubits, targets):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)

    for e in targets:
        for i,val in enumerate(e):
            if val == 0:
                X(wires[i])
        H(wires[-1])
        X.ctrl(num_qubits-1)(wires[:-1], wires[-1])
        H(wires[-1])
        for i,val in enumerate(e):
            if val == 0:
                X(wires[i])
    return routine

my_prog = Program()
e_Uf = economic_oracle(num_qubits, targets)
my_qbits = my_prog.qalloc(num_qubits)
e_Uf(my_qbits)
circuit = my_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
# Exercício 10b
qprog = Program()
Qr = qprog.qalloc(num_qubits)

for q in Qr:
    H(q)
e_Uf(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()

# import one Quantum Processor Unit Factory
from qat.qpus import PyLinalg
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

# Iterate over the final state vector to get all final components
for sample in result:
    print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))

In [ ]:
# Exercício 10c
def economic_diffusion(num_qubits):
    routine = QRoutine()
    wires = routine.new_wires(num_qubits)
    for qubit in wires[:-1]:
        H(qubit)
        X(qubit)
    Z(wires[-1])
    X.ctrl(len(wires)-1)(wires[:-1], wires[-1])
    Z(wires[-1])
    for qubit in wires[:-1]:
        X(qubit)
        H(qubit)

    return routine

my_prog = Program()
e_G = economic_diffusion(num_qubits)
my_qbits = my_prog.qalloc(num_qubits)
e_G(my_qbits)
circuit = my_prog.to_circ()
print("total number of gates: ", len(circuit.ops))
# Display quantum circuit
%qatdisplay circuit --svg

In [ ]:
probs = []
repetitions = []
# Exercício 10d
qprog = Program()
Qr = qprog.qalloc(num_qubits)

for q in Qr:
    H(q)
for i in range(rep):
    e_Uf(Qr)
    e_G(Qr)

# Export this program into a quantum circuit
circuit = qprog.to_circ()
%qatdisplay circuit
# Create a Quantum Processor Unit
linalgqpu = PyLinalg()
# Create a job
job = circuit.to_job()
# Submit the job to the QPU
result = linalgqpu.submit(job)

prob = 0
for sample in result:
    for e in targets:
        if ('|' + ''.join(map(str,e)) in str(sample.state)):
            print("Estado %s: probabilidade %s amplitude %s " % (sample.state, sample.probability, sample.amplitude))
            prob+=sample.probability
print('Probabilidade de sucesso: ',prob)
probs.append(prob)
repetitions.append(i)
print('******************************************************************************')
